In [1]:
import os
import pandas as pd
import yfinance as yf
from pathlib import Path
from datetime import datetime

In [2]:
# Set folder paths relative to notebook location
PROJECT_ROOT = Path("..")          # from notebook/ go up one level
DATA = PROJECT_ROOT / "data"
RAW = DATA / "raw"

# Make sure the raw folder exists
RAW.mkdir(parents=True, exist_ok=True)

In [3]:
# Timestamp helper
def ts():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

# Save function
def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path.resolve())
    return path

# Validation function
def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

In [4]:
# Project-relevant symbols
SYMBOLS = {
    "sp500": "^GSPC",  # S&P 500 index
    "vix": "^VIX"      # VIX index (CBOE Volatility Index)
}

data_frames = {}

# Download data from Yahoo Finance
for name, ticker in SYMBOLS.items():
    df = yf.download(ticker, period="1y", interval="1d").reset_index()[['Date', 'Close']]
    df.columns = ['date','adj_close']
    data_frames[name] = df

    # Save using your save_csv function
    save_csv(
        df.sort_values('date'),
        prefix="market",
        source="yfinance",
        symbol=ticker,
        name=name
    )

# Validate both datasets
for name, df in data_frames.items():
    print(name, validate(df, ["date", "adj_close"]))

/var/folders/m3/srcst_cd347fv_bjsd6nkqyh0000gn/T/ipykernel_33183/1544293442.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", interval="1d").reset_index()[['Date', 'Close']]
[*********************100%***********************]  1 of 1 completed
/var/folders/m3/srcst_cd347fv_bjsd6nkqyh0000gn/T/ipykernel_33183/1544293442.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", interval="1d").reset_index()[['Date', 'Close']]


Saved /Users/yihanyao/bootcamp_yihan_yao/project/data/raw/market_source-yfinance_symbol-^GSPC_name-sp500_20250820_102321.csv


[*********************100%***********************]  1 of 1 completed

Saved /Users/yihanyao/bootcamp_yihan_yao/project/data/raw/market_source-yfinance_symbol-^VIX_name-vix_20250820_102321.csv
sp500 {'missing': [], 'shape': (251, 2), 'na_total': 0}
vix {'missing': [], 'shape': (251, 2), 'na_total': 0}


## Documentation
- **Source**
    - URL/endpoint: yfinance (no direct URL)
    - Ticker: ^GSPC (S&P 500 Index), ^VIX (CBOE Volatility Index) 
    - Params period='1y', interval='1d'
    - Output: Daily adjusted close prices stored as CSV in data/raw/ with timestamped filenames 
- **Assumptions & risks**
    - Yfinance provides sufficient historical market and volatility data for analysis. 
    - Required columns exist and are validated after download.
    - Data is stored locally as CSV with timestamp, ensuring reproducibility
    - Free data source (yfinance) may be subject to outages or delays
    - VIX and S&P 500 availability depends on Yahoo Finance's API backend, which may change without notice
    - CSV schema is simple, but if yfinance changes column names, downstream validation could fail 
- Confirm `.env` is not committed